In [1]:
from scipy.io import loadmat
from scipy import signal
import numpy as np
import os

def calculate_avg_psd(segment, fs, freq_range):
    f, Pxx = signal.welch(segment, fs=fs, nperseg=128)
    freq_indices = np.where((f >= freq_range[0]) & (f <= freq_range[1]))[0]
    avg_psd = np.mean(Pxx[freq_indices])
    return avg_psd

def load_data(data_dir): 
    fs = 128
    fStart = [4, 8, 14, 31]
    fEnd = [7, 13, 30, 45]
    selected_channels = range(32)
    seconds_to_exclude = 10

    datasets_X, datasets_y = [], []

    for filename_data in os.listdir(data_dir):
        if filename_data.endswith(".mat"):
            # Load data and labels
            mat_data = loadmat(os.path.join(data_dir, filename_data))
            data = mat_data['data']
            labels = mat_data['labels']

            for trial_index in range(data.shape[0]):
                dataset_X = []
                trial_data = data[trial_index, selected_channels, :]

                # Exclude the first and last 10 seconds
                start_index = seconds_to_exclude * fs
                end_index = -seconds_to_exclude * fs

                trial_data = trial_data[:, start_index:end_index]

                for band_index, band in enumerate(fStart):
                    b, a = signal.butter(4, [fStart[band_index]/fs, fEnd[band_index]/fs], 'bandpass')
                    filtered_data = signal.filtfilt(b, a, trial_data)
                    features = []

                    for lead in selected_channels:
                        feature = []
                        for de_index in range(0, filtered_data.shape[1] - fs, fs):
                            data_segment = filtered_data[lead, de_index: de_index + fs]
                            avg_psd = calculate_avg_psd(data_segment, fs, [fStart[band_index], fEnd[band_index]])  # Modify the frequency range as needed
                            feature.append(avg_psd)

                        features.append(feature)

                    features = np.array(features)
                    dataset_X.append(features)

                #dataset_X = np.array(dataset_X)
                #dataset_X = dataset_X.reshape((dataset_X.shape[0], dataset_X.shape[1], -1))

                datasets_X.append(dataset_X)
                datasets_y.append(labels[trial_index])

    datasets_X, datasets_y = np.array(datasets_X), np.array(datasets_y)

    return datasets_X, datasets_y

data_dir = "E:/STUDY/Publications/Thesis/Brain Emotion Detection/Dataset/DEAP/data_preprocessed_matlab/"

datasets_X, datasets_y = load_data(data_dir)
print(datasets_X.shape)
print(datasets_y.shape)


(1280, 4, 32, 42)
(1280, 4)


In [2]:
new_datasets_X = np.copy(datasets_X)
new_datasets_X = np.transpose(new_datasets_X, (0, 2, 3, 1))
print(new_datasets_X.shape)

(1280, 32, 42, 4)


In [3]:
np.save('DEAP_PSD_X.npy', new_datasets_X)
np.save('DEAP_PSD_y.npy', datasets_y)